-incarca datele din fisierele .pkl, extrage imaginile si etichetele

-le transforma in vectori numerici si normalizează valorile pixelilor între 0 și 1, pregătind seturile de date pentru antrenarea modelului.

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

def load_extended_mnist(train_file="extended_mnist_train.pkl",
                        test_file="extended_mnist_test.pkl"):
    with open(train_file, "rb") as fp:
        train = pickle.load(fp)
    with open(test_file, "rb") as fp:
        test = pickle.load(fp)

    train_X, train_y = [], []
    for img, lbl in train:
        train_X.append(img.flatten())
        train_y.append(lbl)

    test_X = []
    for img, _ in test:
        test_X.append(img.flatten())

    train_X = np.array(train_X, dtype=np.float32) / 255.0
    train_y = np.array(train_y, dtype=np.int64)
    test_X  = np.array(test_X,  dtype=np.float32) / 255.0
    return train_X, train_y, test_X

-transforma etichetele numerice (ex. 0–9) intr-o reprezentare binara (matrice one-hot), unde fiecare rand are valoarea 1 la pozitia clasei corespunzatoare si 0 in rest

In [3]:
def one_hot_encode(labels, num_classes=None):
    if num_classes is None:
        num_classes = int(labels.max()) + 1
    Y = np.zeros((labels.shape[0], num_classes), dtype=np.float32)
    Y[np.arange(labels.shape[0]), labels] = 1.0
    return Y

-creează parametrii modelului — matricea de greutăți W și vectorul de biase b — inițializate aleator (după regula Xavier) pentru a începe antrenarea rețelei.

In [17]:
def initialize_params(input_dim, output_dim):
    limit = np.sqrt(6.0 / (input_dim + output_dim))
    W = np.random.default_rng(42).uniform(-limit, limit, (input_dim, output_dim)).astype(np.float32)
    b = np.zeros((1, output_dim), dtype=np.float32)
    return W, b

-transformă valorile de ieșire ale modelului în probabilități, asigurând că toate sunt pozitive și însumate la 1 pentru fiecare eșantion

In [6]:
def softmax(Z):
    Z = Z - np.max(Z, axis=1, keepdims=True)
    expZ = np.exp(Z)
    return expZ / np.sum(expZ, axis=1, keepdims=True)

-calculează ieșirea modelului: combină intrările cu greutățile (X·W + b) și aplică funcția softmax pentru a obține probabilitățile pentru fiecare clasă.

In [7]:
def forward_propagation(X, W, b):
    Z = np.dot(X, W) + b
    A = softmax(Z)
    return A

-calculează eroarea medie (cross-entropy) dintre predicțiile modelului și etichetele reale; opțional adaugă un termen de regularizare L2 (weight_decay) pentru a reduce overfittingul.

In [8]:
def compute_loss(A, Y, W=None, weight_decay=0.0):
    m = Y.shape[0]
    eps = 1e-12
    log_likelihood = -np.log(np.clip(A, eps, 1.0)[np.arange(m), Y.argmax(axis=1)])
    ce = np.sum(log_likelihood) / m
    if W is None or weight_decay <= 0:
        return ce
    return ce + 0.5 * weight_decay * np.sum(W * W)

-calculează gradientele (derivatele) față de greutăți (dW) și biase (db), adică direcția în care trebuie ajustați parametrii pentru a reduce eroarea; include și efectul regularizării L2 dacă este activată.

In [9]:
def backward_propagation(X, Y, A, W=None, weight_decay=0.0):
    m = X.shape[0]
    dZ = (A - Y) / m
    dW = np.dot(X.T, dZ)
    if weight_decay > 0 and W is not None:
        dW = dW + weight_decay * W
    db = np.sum(dZ, axis=0, keepdims=True)
    return dW.astype(np.float32), db.astype(np.float32)

-actualizează greutățile și biasele modelului folosind gradient descent; dacă este activat, aplică și momentum pentru a accelera convergența și a evita oscilațiile în procesul de antrenare.

In [10]:
def update_params(W, b, dW, db, learning_rate, momentum_state=None, momentum=0.0):
    if momentum_state is None or momentum <= 0.0:
        W -= learning_rate * dW
        b -= learning_rate * db
        return W, b, None
    vW, vb = momentum_state
    vW = momentum * vW - learning_rate * dW
    vb = momentum * vb - learning_rate * db
    W += vW
    b += vb
    return W, b, (vW, vb)

-realizează antrenarea efectivă a modelului: inițializează parametrii, parcurge datele în mini-loturi (batch-uri), face forward propagation, calculează eroarea și gradientele, apoi actualizează parametrii la fiecare pas. Repetă procesul pentru mai multe epoci, ajustând treptat learning rate-ul și aplicând momentum sau regularizare dacă sunt setate.

In [ ]:
def train_model(train_X, train_Y, input_dim, output_dim,
                epochs=100, learning_rate=0.01, batch_size=100,
                momentum=0.9, weight_decay=3e-4, lr_decay=1.0,
                verbose=True):
    W, b = initialize_params(input_dim, output_dim)
    momentum_state = (np.zeros_like(W), np.zeros_like(b)) if momentum > 0 else None

    for epoch in range(epochs):
        perm = np.random.permutation(train_X.shape[0])
        train_X = train_X[perm]
        train_Y = train_Y[perm]

        for i in range(0, train_X.shape[0], batch_size):
            X_batch = train_X[i:i+batch_size]
            Y_batch = train_Y[i:i+batch_size]

            A = forward_propagation(X_batch, W, b)
            loss = compute_loss(A, Y_batch, W, weight_decay)
            dW, db = backward_propagation(X_batch, Y_batch, A, W, weight_decay)
            W, b, momentum_state = update_params(W, b, dW, db, learning_rate,
                                                 momentum_state, momentum)

        if verbose:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')
        learning_rate *= lr_decay

    return W, b

-calculează ieșirile modelului pentru datele de intrare și returnează clasa cu probabilitatea cea mai mare pentru fiecare exemplu (predicția finală).

In [12]:
def predict(X, W, b):
    A = forward_propagation(X, W, b)
    return np.argmax(A, axis=1)

-această parte încarcă datele de antrenare și test, apoi împarte setul de antrenare în 90% pentru antrenare și 10% pentru validare, păstrând proporția claselor. În final, etichetele de antrenare sunt transformate în reprezentare one-hot, necesară pentru calculul corect al erorii în timpul antrenării

In [ ]:
train_X, train_y, test_X = load_extended_mnist(
    train_file="fii-nn-2025-homework-2/extended_mnist_train.pkl",
    test_file="fii-nn-2025-homework-2/extended_mnist_test.pkl"
)

rng = np.random.default_rng(42)
idx = np.arange(len(train_y))
val_mask = np.zeros_like(idx, dtype=bool)
for c in range(train_y.max()+1):
    ids = idx[train_y == c]
    rng.shuffle(ids)
    k = max(1, int(0.10 * len(ids)))
    val_mask[ids[:k]] = True
X_tr, y_tr = train_X[~val_mask], train_y[~val_mask]
X_val, y_val = train_X[val_mask], train_y[val_mask]

num_classes = int(train_y.max()) + 1
train_Y_oh = one_hot_encode(y_tr, num_classes)


-setează hiper-parametrii antrenării: dimensiunea intrării și ieșirii, numărul de epoci, rata de învățare, mărimea batch-ului, momentum, regularizarea L2 (weight_decay) și scăderea treptată a LR (lr_decay). Acestea controlează cum și cât de repede învață modelul.

In [14]:
input_dim = train_X.shape[1]
output_dim = num_classes
epochs = 40
learning_rate = 0.25
batch_size = 256
momentum = 0.9
weight_decay = 3e-4
lr_decay = 0.96

-această parte antrenează modelul folosind funcția train_model() cu hiperparametrii definiți, apoi face predicții pe seturile de antrenare și validare pentru a calcula și afișa acuratețea modelului — adică cât de bine a învățat să clasifice imaginile.

In [ ]:
W, b = train_model(X_tr, train_Y_oh, input_dim, output_dim,
                   epochs=epochs, learning_rate=learning_rate, batch_size=batch_size,
                   momentum=momentum, weight_decay=weight_decay, lr_decay=lr_decay,
                   verbose=True)

train_pred = predict(X_tr, W, b)
val_pred = predict(X_val, W, b)
print(f'Training Accuracy: {accuracy_score(y_tr, train_pred) * 100:.2f}%')
print(f'Validation Accuracy: {accuracy_score(y_val, val_pred) * 100:.2f}%')

Epoch 1/40, Loss: 0.3353
Epoch 2/40, Loss: 0.2693
Epoch 3/40, Loss: 0.3449
Epoch 4/40, Loss: 0.2991
Epoch 5/40, Loss: 0.3219
Epoch 6/40, Loss: 0.3033
Epoch 7/40, Loss: 0.3888
Epoch 8/40, Loss: 0.2997
Epoch 9/40, Loss: 0.2523
Epoch 10/40, Loss: 0.3647
Epoch 11/40, Loss: 0.3716
Epoch 12/40, Loss: 0.3612
Epoch 13/40, Loss: 0.3586
Epoch 14/40, Loss: 0.3074
Epoch 15/40, Loss: 0.3501
Epoch 16/40, Loss: 0.4127
Epoch 17/40, Loss: 0.2928
Epoch 18/40, Loss: 0.3418
Epoch 19/40, Loss: 0.3774
Epoch 20/40, Loss: 0.2686
Epoch 21/40, Loss: 0.2619
Epoch 22/40, Loss: 0.2619
Epoch 23/40, Loss: 0.3864
Epoch 24/40, Loss: 0.2920
Epoch 25/40, Loss: 0.2418
Epoch 26/40, Loss: 0.2415
Epoch 27/40, Loss: 0.4098
Epoch 28/40, Loss: 0.3063
Epoch 29/40, Loss: 0.3237
Epoch 30/40, Loss: 0.3258
Epoch 31/40, Loss: 0.2635
Epoch 32/40, Loss: 0.3213
Epoch 33/40, Loss: 0.2584
Epoch 34/40, Loss: 0.2752
Epoch 35/40, Loss: 0.2980
Epoch 36/40, Loss: 0.2686
Epoch 37/40, Loss: 0.2830
Epoch 38/40, Loss: 0.2979
Epoch 39/40, Loss: 0.

-această parte re-antrenează modelul pe întreg setul de date pentru performanță maximă, apoi generează predicțiile finale pentru imaginile de test. Rezultatele sunt salvate într-un fișier submission.csv, în formatul cerut (ID, target), pentru a fi trimise spre evaluare.

In [16]:
Y_full_oh = one_hot_encode(train_y, num_classes)
W_final, B_final = train_model(train_X, Y_full_oh, input_dim, output_dim,
                               epochs=28, learning_rate=0.20, batch_size=batch_size,
                               momentum=momentum, weight_decay=weight_decay, lr_decay=lr_decay,
                               verbose=False)

test_predictions = predict(test_X, W_final, B_final)

predictions_csv = {"ID": np.arange(len(test_predictions), dtype=int),
                   "target": test_predictions.astype(int)}
pd.DataFrame(predictions_csv).to_csv("submission.csv", index=False)
print("Saved submission.csv")

Saved submission.csv
